In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# SK-learn libraries for learning.
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.utils import resample
from collections import Counter


# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc


# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

# SK-learn libraries for model selection 
from sklearn.model_selection import train_test_split

# json libraries to parse json file
import json
from pandas.io.json import json_normalize

import nltk
from apyori import apriori
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import wordnet 
from nltk.corpus import sentiwordnet as swn
from collections import Counter
from textblob import TextBlob

In [10]:
cd

/home/joanna_huang


In [11]:
cd W266-Final-Project/ 

/home/joanna_huang/W266-Final-Project


In [2]:
cd tripadvisor/reviews/csv

/home/joanna_huang/W266-Final-Project/tripadvisor/reviews/csv


In [3]:
filepath = ['Chicago_Illinois_1.csv','Chicago_Illinois_2.csv', 'Las_Vegas_Nevada_1.csv', 'Las_Vegas_Nevada_2.csv',
'Las_Vegas_Nevada_3.csv','Las_Vegas_Nevada_4.csv','Las_Vegas_Nevada_5.csv', 'Las_Vegas_Nevada_6.csv',
'Las_Vegas_Nevada_7.csv', 'New_York_City_New_York_1.csv', 'New_York_City_New_York_2.csv', 
'New_York_City_New_York_3.csv', 'New_York_City_New_York_4.csv', 'New_York_City_New_York_5.csv',
'New_York_City_New_York_6.csv', 'New_York_City_New_York_7.csv', 'San_Francisco_California_1.csv',
'San_Francisco_California_2.csv']

data = pd.DataFrame()
for path in filepath:
    df = pd.read_csv(path)
    df['city'] = path.split('_')[0]
    data = data.append(df)
data.shape #(968772, 19)
data = data.reset_index(drop=True)
del data['neighborhood'] #remove column given 80+% null
data['review_body'] = [x.lower() for x in data['review_body']]
data['review_body'] = [x.split('.') for x in data['review_body']]

/home/joanna_huang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
cd

/home/joanna_huang


In [5]:
cd W266-Final-Project/ 

/home/joanna_huang/W266-Final-Project


In [6]:
with open('NRC-Emotion-Lexicon-Wordlevel-v0.92.csv', 'r') as f:
    df = pd.read_csv(f, delimiter='\t', header=0, names=['word', 'emotion', 'relation'])

# Create lists of emotion words
#(anger, fear, anticipation, trust, surprise, sadness, joy, and disgust)
anticipation = list(df[(df['emotion']=='anticipation') & (df['relation']!=0)]['word'])
anger = list(df[(df['emotion']=='anger') & (df['relation']!=0)]['word'])
fear = list(df[(df['emotion']=='fear') & (df['relation']!=0)]['word'])
trust = list(df[(df['emotion']=='trust') & (df['relation']!=0)]['word'])
surprise = list(df[(df['emotion']=='surprise') & (df['relation']!=0)]['word'])
sadness = list(df[(df['emotion']=='sadness') & (df['relation']!=0)]['word'])
joy = list(df[(df['emotion']=='joy') & (df['relation']!=0)]['word'])
disgust = list(df[(df['emotion']=='disgust') & (df['relation']!=0)]['word'])
neg_NRC = list(df[(df['emotion']=='negative') & (df['relation']!=0)]['word'])
pos_NRC = list(df[(df['emotion']=='positive') & (df['relation']!=0)]['word'])

In [ ]:
all_sent = []
count = 515120
for item in data['review_body'][515120:]:
    print(count)
    count+=1
    tagged_sent = []
    for sent in item:
        tokenized = nltk.word_tokenize(sent)
        tagged=nltk.pos_tag(tokenized)
        tagged_sent.append(tagged)
    all_sent.append(tagged_sent)

### Extract Topic Features

In [60]:
Service= ['service', 'services', 'server', 'waiter', 'staff', 'front', 'desk', 'valet', \
          'checkin', 'check-in', 'check', 'management', 'bellhop', 'maintenance', 'concierge',\
          'reception', 'wait', 'booking', 'checkout', 'reservation', 'clerk', 'manager', \
          'representative', 'customer']
Accessibility=['subway', 'accessible', 'access', 'accessibility', 'location', 'area', \
               'walk', 'walking', 'neighborhood', 'handicap', 'distance', 'wheelchair', \
               'airport', 'proximity', 'downtown', 'metro']
Room= ['room', 'view', 'rooms', 'bed', 'bathroom', 'tub', 'phone', 'door', 'lock', 'telephone', \
       'shower', 'TV', 'bathtub', 'microwave', 'fridge', 'key', 'TV', 'bedroom', 'wall', \
       'bedding', 'housekeeping', 'mini-fridge', 'mattress', 'carpet', 'wallpaper', 'toilet', \
       'sofa', 'lamp', 'bedside']
Attractions=['shopping', 'park', 'shop', 'dining', 'dine', 'lake', 'city', 'restaurant', 'bar', \
             'pier', 'center', 'centre', 'show', 'entertainment', 'concert', 'amusement',
            'monument', 'tourist']
Amenities=['coffee', 'breakfast', 'wifi', 'facilities', 'pool', 'pools', 'facility', 'elevator', \
           'equipment', 'AC', 'condition', 'air', 'conditioning', 'ice', 'lounge', 'internet', \
           'cleaning', 'laundry', 'minibar', 'free', 'buffet', 'fitness', 'gym']
Member=['VIP', 'loyalty', 'platinum', 'select']
Money=['discount', 'deal', 'budget', 'rate', 'billing', 'bill', 'payment']
Hotel=['property', 'lobby',  'decor', 'style', 'hallway', 'furniture', 'building']

In [10]:
import textstat

In [61]:
reviews = []
sentences = []
word_count =[]
service = []
service_words=[]
access = []
access_words=[]
room = []
room_words=[]
attractions =[]
att_words=[]
amenity=[]
amenity_words=[]
membership=[]
member_words=[]
money=[]
money_words=[]
hotel=[]
hotel_words=[]
readability_ease=[]
readability_grade=[]

untagged_sent = []
count=515120

In [ ]:
for rev in all_sent:
    review_no = data.index.values[count]
    print(review_no)
    count+=1
    print(count)
    #print(rev)
    for sent in rev:
        if len(sent)==0:
            pass
        else:
            #print(sent)
            reviews.append(review_no)
            # Append sentence
            sentences.append(sent)
            # Get word count
            word_count.append(len(sent))
            count_service = Counter(x[0] for x in sent if x[0] in Service)
            service.append(sum(count_service.values()))
            service_words.append(count_service.keys())
            count_access = Counter(x[0] for x in sent if x[0] in Accessibility)
            access.append(sum(count_access.values()))
            access_words.append(count_access.keys())
            count_room = Counter(x[0] for x in sent if x[0] in Room)
            room.append(sum(count_room.values()))
            room_words.append(count_room.keys())
            count_att = Counter(x[0] for x in sent if x[0] in Attractions)
            attractions.append(sum(count_att.values()))
            att_words.append(count_att.keys())
            count_amenity = Counter(x[0] for x in sent if x[0] in Amenities)
            amenity.append(sum(count_amenity.values()))
            amenity_words.append(count_amenity.keys())
            count_member = Counter(x[0] for x in sent if x[0] in Member)
            membership.append(sum(count_member.values()))
            member_words.append(count_member.keys())
            count_money = Counter(x[0] for x in sent if x[0] in Money)
            money.append(sum(count_money.values()))
            money_words.append(count_money.keys())
            count_hotel = Counter(x[0] for x in sent if x[0] in Hotel)
            hotel.append(sum(count_hotel.values()))
            hotel_words.append(count_hotel.keys())

            
            new_sent = " ".join([a for a, b in sent])
            untagged_sent.append(new_sent)
            readability_ease.append(textstat.flesch_reading_ease(new_sent))
            readability_grade.append(textstat.flesch_kincaid_grade(new_sent))
            
            
print(count)

In [63]:
# Create a dataframe of sentence-level records
setup = {'Reviews':reviews, 'Sentence':sentences, 'Untagged': untagged_sent, \
         'Word_Count': word_count, 'Service': service, 'Service Words': service_words,\
         'Access': access, 'Access Words': access_words, 'Room': room,\
         'Room Words': room_words, 'Attractions': attractions, 'Att Words': att_words,\
         'Amenity': amenity, 'Amenity Words': amenity_words, 'Member': membership,\
         'Member Words': member_words, 'Money': money, 'Money Words': money_words,\
         'Hotel': hotel, 'Hotel Words': hotel_words, 'readability_ease': readability_ease,\
         'readability_grade': readability_grade
}
opinions = pd.DataFrame(setup)  


In [68]:
cd W266-Final-Project/ 

/home/joanna_huang/W266-Final-Project


In [69]:
#opinions.to_csv('topics_secondhalf')
opinions1 = pd.read_csv('topics_firsthalf')
opinions = pd.read_csv('topics_secondhalf')
opinions_all = pd.concat([opinions1,opinions])

/home/joanna_huang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


In [71]:
# Aggregate sentence-level dataframe back into review-level
aggregation_functions = {'Reviews':'sum', 'Sentence':'sum', 'Untagged': 'sum', \
         'Word_Count': 'sum', 'Service': 'sum', \
         'Access': 'sum', 'Room': 'sum','Attractions': 'sum', 
         'Amenity': 'sum', 'Member': 'sum', 'Money': 'sum', 
         'Hotel': 'sum', 'readability_ease': 'sum',\
         'readability_grade': 'sum'}
df1 = opinions_all.groupby(opinions_all['Reviews']).agg(aggregation_functions)
# Make values percentages
df2 = df1[['Service', 'Room', 'Access', 'Attractions', 'Amenity', 'Member',\
          'Money', 'Hotel']].div(df1.Word_Count, axis=0)
df2['readability_ease'] = df1['readability_ease']
df2['readability_grade'] = df1['readability_grade']
df2['Word_Count'] = df1['Word_Count']

In [74]:
print(df2.head(2))
print(len(df2))

          Service     Room    Access  Attractions  Amenity  Member  Money  \
Reviews                                                                     
0        0.000000  0.00000  0.015873     0.031746      0.0     0.0    0.0   
1        0.015625  0.03125  0.015625     0.046875      0.0     0.0    0.0   

            Hotel  readability_ease  readability_grade  Word_Count  
Reviews                                                             
0        0.000000            193.80               28.2          63  
1        0.015625            319.09               23.2          64  
1030236


In [75]:
cd

/home/joanna_huang


In [2]:
cd LIWC

/home/joanna_huang/W266-Final-Project/LIWC


In [3]:
df = pd.read_csv('df2.fixed')

# LIWC percentages are *100
df['affect'] = df['affect']*0.01
df['posemo'] = df['posemo']*0.01
df['negemo'] = df['negemo']*0.01
df['anx'] = df['anx']*0.01
df['anger'] = df['anger']*0.01
df['sad'] = df['sad']*0.01
df['cogproc'] = df['cogproc']*0.01
df['insight'] = df['insight']*0.01
df['cause'] = df['cause']*0.01
df['discrep'] = df['discrep']*0.01
df['tentat'] = df['tentat']*0.01
df['certain'] = df['certain']*0.01
df['differ'] = df['differ']*0.01

In [82]:
df['Service'] = df2['Service']
df['Access'] = df2['Access']
df['Room'] = df2['Room']
df['Attractions'] = df2['Attractions']
df['Amenity'] = df2['Amenity']
df['Member'] = df2['Member']
df['Money'] = df2['Money']
df['Hotel'] = df2['Hotel']
df['readability_grade'] = df2['readability_grade']
df['readability_ease'] = df2['readability_ease']

### Check correlation

In [83]:
df_corr = df[['Word_Count','Helpful', 'city', 'class', 'activity_level',
         'Third Person', 'First Person', 'negemo', 'affect', 'Joy', 'Disgust','Sad',
         'Narrative Seq', 'Past Tense','spatial', 'temporal','Polarity2', 'cogproc', 'Comparatives',
        'money', 'Service', 'Access', 'Room', 'Attractions', 'Amenity', 'Member', 'Money', 'Hotel', 
                     'readability_grade', 'readability_ease']]
# calculate the correlation matrix
corr = df_corr.corr()
corr.style.background_gradient().set_precision(2)

,Word_Count,Helpful,city,class,activity_level,Third Person,First Person,negemo,affect,Joy,Disgust,Sad,Narrative Seq,Past Tense,spatial,temporal,Polarity2,cogproc,Comparatives,money,Service,Access,Room,Attractions,Amenity,Member,Money,Hotel,readability_grade,readability_ease
Word_Count,1,0.12,0.024,0.022,0.12,0.15,0.1,0.054,-0.16,-0.26,0.045,0.0047,0.15,0.093,0.17,0.34,-0.26,0.083,0.014,-0.036,-0.13,-0.17,-0.051,-0.096,-0.039,0.014,0.03,-0.025,0.97,0.81
Helpful,0.12,1,0.00011,0.019,0.42,0.0073,-0.011,0.00092,-0.028,-0.049,-0.0048,0.003,-0.00073,-0.023,0.019,0.043,-0.039,0.0066,0.012,0.0047,-0.043,-0.014,0.003,-0.0026,0.016,0.016,0.012,0.02,0.12,0.11
city,0.024,0.00011,1,0.042,-0.0014,0.046,-0.019,0.014,-0.11,-0.037,0.064,0.17,0.029,0.015,0.018,0.055,-0.061,0.057,0.023,0.2,-0.095,-0.2,-0.019,-0.17,0.042,0.001,0.0022,-0.072,-0.0068,0.05
class,0.022,0.019,0.042,1,0.022,-0.024,0.017,-0.029,0.012,0.038,-0.051,-0.024,-0.0052,-0.042,0.045,0.0051,0.12,-0.0093,-0.018,-0.029,0.041,-0.1,-0.013,0.076,-0.093,0.016,-0.032,0.024,0.042,-0.0024
activity_level,0.12,0.42,-0.0014,0.022,1,0.0078,-0.037,-0.0064,-0.045,-0.075,-0.016,0.0054,-0.011,-0.05,0.027,0.051,-0.055,0.0061,0.022,0.013,-0.076,-0.0054,0.022,0.0087,0.041,0.016,0.019,0.041,0.12,0.13
Third Person,0.15,0.0073,0.046,-0.024,0.0078,1,0.051,0.055,-0.078,-0.14,0.063,0.017,0.12,0.12,0.025,0.14,-0.15,0.1,-0.0043,-0.017,-0.073,-0.16,-0.1,-0.12,-0.08,1.7e-05,0.018,-0.055,0.11,0.14
First Person,0.1,-0.011,-0.019,0.017,-0.037,0.051,1,-0.013,-0.018,-0.039,-0.032,-0.043,0.12,0.32,-0.00073,0.082,-0.002,-0.035,-0.04,-0.13,-0.00042,-0.093,-0.021,-0.066,-0.081,-0.0094,-0.013,-0.07,0.082,0.066
negemo,0.054,0.00092,0.014,-0.029,-0.0064,0.055,-0.013,1,0.2,-0.11,0.25,0.18,0.051,0.062,-0.038,-0.009,-0.25,0.045,0.02,0.023,0.00049,-0.078,0.031,-0.065,-0.0042,0.0048,0.012,-0.0079,0.052,0.045
affect,-0.16,-0.028,-0.11,0.012,-0.045,-0.078,-0.018,0.2,1,0.28,-0.049,-0.013,-0.086,-0.054,-0.27,-0.35,0.27,-0.085,-0.02,-0.033,0.096,0.11,-0.0088,0.064,0.047,-0.0077,-0.022,0.013,-0.13,-0.13
Joy,-0.26,-0.049,-0.037,0.038,-0.075,-0.14,-0.039,-0.11,0.28,1,-0.13,-0.036,-0.15,-0.1,-0.099,-0.22,0.44,-0.1,-0.066,0.024,0.16,0.13,-0.045,0.14,0.035,-0.019,-0.0042,-0.00058,-0.22,-0.2


#df.sample.astype(bool).sum(axis=0)

# Model Iterations

In [4]:
y = df['Rating']
# Controls: word count, helpful, city, class, popularity, activity level

X = df[['Word_Count','Helpful', 'city', 'class', 'activity_level',
         'Third Person', 'First Person', 'negemo', 'affect', 'Joy', 'Disgust','Sad',
         'Narrative Seq', 'Past Tense','spatial', 'temporal','Polarity2', 'cogproc']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [ ]:
# Multinomial Logistic Regression
#mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(balanced_x, balanced_y)
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(X_train, y_train)
y_pred_mul = mul_lr.predict(X_test)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred_mul)
print(confusion_matrix)
print(metrics.classification_report(y_test, y_pred_mul))
print(metrics.accuracy_score(y_test, y_pred_mul))

- 0.5894 X = df2[['Word_Count','Helpful', 'city', 'class', 'activity_level',
         'Third Person', 'First Person', 'negemo', 'affect', 'Joy', 'Disgust','Sad',
         'Narrative Seq', 'Past Tense','spatial', 'temporal','Polarity2', 'cogproc']] 

- 0.5897 +'Comparatives'
- 0.5850 + 'readability' (high VIF)
- 0.5946 +'Comparatives','money', 'affiliation' (high VIF with affiliation)
        -0.3222 with downsampled dataset
- 0.5924 +'Service', 'Access', 'Room', 'Attractions', 'Amenity'
- 0.5931 + 'Service', 'Access', 'Room', 'Attractions', 'Amenity', 'Member', 'Money', 'Hotel'
- 0.5953 +'Comparatives''money''Service', 'Access', 'Room', 'Attractions', 'Amenity', 'Member', 'Money', 'Hotel'   
- 0.5913 +'Comparatives''money'
- 0.5897 +'Comparatives'

- 0.5889 Replace Joy with Trust

- 0.581 (half data)  X = df.sample[['Word_Count','Helpful', 'city', 'class', 'activity_level', 'Third Person', 'First Person', 'negemo', 'affect', 'Trust', 'Disgust','Sad', Narrative Seq', 'Past Tense','spatial', 'temporal','Polarity2', 'cogproc']]

- 0.58744 (half data)  +'Service', 'Access', 'Room', 'Attractions', 'Amenity', 'Member', 'Money', 'Hotel', 'readability_grade', 'readability_ease'
- 0.5860 (half data) +'Service', 'Access', 'Room', 'Attractions', 'Amenity'